In [ ]:
# Example of training a linear model that rolls out predictions day-by-day

# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [ ]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [ ]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

In [ ]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 error_bad_lines=False)

In [ ]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [ ]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [ ]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [ ]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing']
df = df[id_cols + cases_col + npi_cols]

In [ ]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [ ]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [ ]:
df

In [ ]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [ ]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [ ]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

In [ ]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

In [ ]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

In [ ]:
# Save model to file
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

First, a sample evaluation data set is created of the form that is given to the predictor.

Second, the predictor is evaluated on this data set, and a resulting predictions file is produced.

### Create sample evaluation data

In [ ]:
# Create hypothetical evaluation data
nb_eval_days = 31
test_df = pd.read_csv(DATA_URL, 
                      parse_dates=['Date'],
                      encoding="ISO-8859-1",
                      error_bad_lines=False)

# Pull out relevant evaluation days
test_df = test_df[(test_df.Date > HYPOTHETICAL_SUBMISSION_DATE) & \
                  (test_df.Date <= HYPOTHETICAL_SUBMISSION_DATE + nb_eval_days)]

# Only include columns we would see during evaluation
test_df = test_df[['CountryName', 'RegionName', 'Date'] + npi_cols]

# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    test_df.update(test_df.groupby(['CountryName', 'RegionName'])[npi_col].ffill().fillna(0))

In [ ]:
# test_df is now in the form of input to a predictor during evaluation
test_df

### Apply predictor to the evaluation data

In [ ]:
def predict(start_date: str, end_date: str, path_to_ips_file: str):
    """
    Generates a file with daily new cases predictions for the given countries, regions and npis, between
    start_date and end_date, included.
    :param start_date: day from which to start making predictions, as a string, format YYYY-MM-DDD
    :param end_date: day on which to stop making predictions, as a string, format YYYY-MM-DDD
    :param path_to_ips_file: path to a csv file containing the intervention plans between start_date and end_date
    :return: Nothing. Saves a csv file called 'start_date_end_date.csv'
    with columns "CountryName,RegionName,Date,PredictedDailyNewCases"
    """
    
    # Add RegionID column that combines CountryName and RegionName for easier manipulation of data\n",
    test_df['GeoID'] = test_df['CountryName'] + '__' + test_df['RegionName'].astype(str)

    # Copy the test data frame
    pred_df = test_df[id_cols].copy()
    # Keep only the requested prediction period.
    # Note: this period *might* be in the future, and pred_df doesn't necessarily contain the requested rows
    pred_df = pred_df[(pred_df.Date >= start_date) & (pred_df.Date <= end_date)]

    # Load historical data to use in making predictions in the same way 
    # df is loaded above to make training data (just copy the reference here for simplicity)
    hist_df = df
    
    # Load model
    with open('model.pkl', 'rb') as model_file:
        model = pickle.load(model_file)
        
    # Make predictions for each country,region pair
    geo_pred_dfs = []
    for g in test_df.GeoID.unique():
        print('\nPredicting for', g)

        # Pull out all relevant data for country c
        hist_gdf = hist_df[hist_df.GeoID == g]
        test_gdf = test_df[test_df.GeoID == g]
        past_cases = np.array(hist_gdf[cases_col])
        past_npis = np.array(hist_gdf[npi_cols])
        future_npis = np.array(test_gdf[npi_cols])

        # Make prediction for each day
        geo_preds = []
        nb_days_to_predict = len(future_npis)
        for days_ahead in range(nb_days_to_predict):

            # Prepare data
            X_cases = past_cases[-nb_lookback_days:]
            X_npis = past_npis[-nb_lookback_days:]
            X = np.concatenate([X_cases.flatten(),
                                X_npis.flatten()])

            # Make the prediction (reshape so that sklearn is happy)
            pred = model.predict(X.reshape(1, -1))[0]
            pred = max(0, pred) # Do not allow predicting negative cases
            geo_preds.append(pred)
            print(pred)
            
            # Append the prediction and npi's for next day
            # in order to rollout predictions for further days.
            past_cases = np.append(past_cases, pred)
            past_npis = np.append(past_npis, future_npis[days_ahead:days_ahead+1], axis=0)

        # Create geo_pred_df with pred column
        geo_pred_df = test_gdf[id_cols].copy()
        geo_pred_df['PredictedDailyNewCases'] = geo_preds
        geo_pred_dfs.append(geo_pred_df)

    # Combine all predictions into a single dataframe
    pred_df = pd.concat(geo_pred_dfs)
    
    # Drop GeoID column to match expected output format
    pred_df = pred_df.drop(columns=['GeoID'])
    pred_df
    
    # Write predictions to csv
    # Save to expected file name
    output_file_name = start_date + "_" + end_date + ".csv"
    pred_df.to_csv(output_file_name, index=None)
    print(f"Predictions saved to {output_file_name}")


In [ ]:
start_date = "2020-08-01"
end_date = "2020-08-31"

In [ ]:
%%time
predict(start_date, end_date, path_to_ips_file="../validation/data/2020-08-01_2020-08-31_ip.csv")

In [ ]:
# Check that predictions are written correctly
!head 2020-08-01_2020-08-31.csv

In [ ]:
# Check the pediction file is valid
from validation.validation import validate_submission

errors = validate_submission(start_date, end_date, "2020-08-01_2020-08-31.csv")
if errors:
    for error in errors:
        print(error)
else:
    print("All good!")

# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

## 4 days

In [ ]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../validation/data/2020-08-01_2020-08-04_ip.csv

In [ ]:
# Check the pediction file is valid
from validation.validation import validate_submission

errors = validate_submission("2020-08-01", "2020-08-04", "2020-08-01_2020-08-04.csv")
if errors:
    for error in errors:
        print(error)
else:
    print("All good!")

## 1 month

In [ ]:
!python predict.py -s 2020-08-01 -e 2020-08-31 -ip ../../validation/data/2020-08-01_2020-08-31_ip.csv

In [ ]:
# Check the pediction file is valid
errors = validate_submission("2020-08-01", "2020-08-31", "2020-08-01_2020-08-31.csv")
if errors:
    for error in errors:
        print(error)
else:
    print("All good!")

## 2 months

In [ ]:
%%time
!python predict.py -s 2020-08-01 -e 2020-09-30 -ip ../../validation/data/2020-08-01_2020-09-30_ip.csv

In [ ]:
# Check the pediction file is valid
errors = validate_submission("2020-08-01", "2020-09-30", "2020-08-01_2020-09-30.csv")
if errors:
    for error in errors:
        print(error)
else:
    print("All good!")